In [14]:
from lite_llm import LiteLLMService, LiteLLMEmbeddingInput, LiteLLMSetting 
from pydantic import HttpUrl, SecretStr

In [15]:
litellm_service=LiteLLMService(
    litellm_setting=LiteLLMSetting(
        url=HttpUrl("http://localhost:9510"),
        token=SecretStr("abc123"),
        model="gemini-2.5-flash",
        frequency_penalty=0.0,
        n=1,
        temperature=0.0,
        top_p=1.0,
        max_completion_tokens=10000,
        dimension=1536,
        embedding_model="gemini-embedding"
    )
)
litellm_service

LiteLLMService(litellm_setting=LiteLLMSetting(url=HttpUrl('http://localhost:9510/'), token=SecretStr('**********'), model='gemini-2.5-flash', frequency_penalty=0.0, n=1, temperature=0.0, top_p=1.0, max_completion_tokens=10000, dimension=1536, embedding_model='gemini-embedding'), async_client=None, client=None)

In [16]:
output = await litellm_service.embedding_llm_async(
    inputs=LiteLLMEmbeddingInput(
        text="Hello, world!"
    )
)

In [17]:
len(output.embedding)

1536

In [18]:
options = [
    "By explicitly computing feature vectors in a higher-dimensional space and then applying a linear SVM.",
    "By replacing the inner product of mapped feature vectors with a kernel function, avoiding explicit computation in high-dimensional space.",  # correct
    "By directly adjusting the weights `w` and bias `b` to create a non-linear decision boundary in the input space.",
    "By reducing the dimensionality of the input data before applying a linear SVM."
]

import numpy as np

# Get embeddings for each option
option_embeddings = []
for option in options:
    embedding_output = await litellm_service.embedding_llm_async(
        inputs=LiteLLMEmbeddingInput(text=option)
    )
    option_embeddings.append(np.array(embedding_output.embedding))

print(f"Generated {len(option_embeddings)} embeddings, each with dimension {len(option_embeddings[0])}")

Generated 4 embeddings, each with dimension 1536


In [19]:
import numpy as np
from typing import List

def cosine_similarity(vector1: np.ndarray, vector2: np.ndarray) -> float:
    """
    Calculate cosine similarity between two vectors.
    
    Args:
        vector1: First vector as numpy array
        vector2: Second vector as numpy array
    
    Returns:
        Cosine similarity value between -1 and 1
    """
    # Calculate dot product
    dot_product = np.dot(vector1, vector2)
    
    # Calculate magnitudes (L2 norms)
    magnitude1 = np.linalg.norm(vector1)
    magnitude2 = np.linalg.norm(vector2)
    
    # Avoid division by zero
    if magnitude1 == 0 or magnitude2 == 0:
        return 0.0
    
    # Calculate cosine similarity
    similarity = dot_product / (magnitude1 * magnitude2)
    
    return similarity

def compute_similarity_matrix(texts: List[str], embeddings: List[np.ndarray]) -> np.ndarray:
    """
    Compute a similarity matrix for a list of text embeddings.
    
    Args:
        texts: List of text strings
        embeddings: List of embedding vectors
    
    Returns:
        NxN similarity matrix where N is the number of texts
    """
    n = len(embeddings)
    similarity_matrix = np.zeros((n, n))
    
    for i in range(n):
        for j in range(n):
            similarity_matrix[i, j] = cosine_similarity(embeddings[i], embeddings[j])
    
    return similarity_matrix

In [20]:
# Compute the 4x4 similarity matrix
similarity_matrix = compute_similarity_matrix(options, option_embeddings)

print("4x4 Cosine Similarity Matrix:")
print("=" * 50)

# Print matrix with proper formatting
for i in range(len(options)):
    row_str = ""
    for j in range(len(options)):
        row_str += f"{similarity_matrix[i, j]:.4f}  "
    print(f"Option {i+1}: {row_str}")

print("\nDetailed Matrix with Labels:")
print("=" * 80)

# Print with option labels for clarity
for i, option1 in enumerate(options):
    print(f"\nOption {i+1}: {option1[:50]}...")
    for j, option2 in enumerate(options):
        print(f"  vs Option {j+1}: {similarity_matrix[i, j]:.4f}")

# Also display as a clean numpy array
print(f"\nNumpy Array:\n{similarity_matrix}")
print(f"\nMatrix Shape: {similarity_matrix.shape}")

4x4 Cosine Similarity Matrix:
Option 1: 1.0000  0.8497  0.6905  0.7976  
Option 2: 0.8497  1.0000  0.6641  0.7329  
Option 3: 0.6905  0.6641  1.0000  0.6729  
Option 4: 0.7976  0.7329  0.6729  1.0000  

Detailed Matrix with Labels:

Option 1: By explicitly computing feature vectors in a highe...
  vs Option 1: 1.0000
  vs Option 2: 0.8497
  vs Option 3: 0.6905
  vs Option 4: 0.7976

Option 2: By replacing the inner product of mapped feature v...
  vs Option 1: 0.8497
  vs Option 2: 1.0000
  vs Option 3: 0.6641
  vs Option 4: 0.7329

Option 3: By directly adjusting the weights `w` and bias `b`...
  vs Option 1: 0.6905
  vs Option 2: 0.6641
  vs Option 3: 1.0000
  vs Option 4: 0.6729

Option 4: By reducing the dimensionality of the input data b...
  vs Option 1: 0.7976
  vs Option 2: 0.7329
  vs Option 3: 0.6729
  vs Option 4: 1.0000

Numpy Array:
[[1.         0.84968432 0.69050761 0.79757521]
 [0.84968432 1.         0.66406493 0.73291624]
 [0.69050761 0.66406493 1.         0.67288202]
 

In [1]:
import chromadb
from lite_llm import LiteLLMEmbeddingInput, LiteLLMService, LiteLLMSetting
from pydantic import HttpUrl, SecretStr

litellm_setting=LiteLLMSetting(
    url=HttpUrl("http://localhost:9510"),
    token=SecretStr("abc123"),
    model="gemini-2.5-flash",
    frequency_penalty=0.0,
    n=1,
    temperature=0.0,
    top_p=1.0,
    max_completion_tokens=10000,
    dimension=1536,
    embedding_model="gemini-embedding"
)

litellm_service = LiteLLMService(litellm_setting=litellm_setting)
client = chromadb.PersistentClient(path="./chroma_database")
collection = client.get_or_create_collection(name="questions")

In [10]:
import json 
from uuid import uuid4
from tqdm import tqdm
import time 


with open("/home/vuiem/KLTN/services/generation/src/generation/shared/static_files/ml_mcqs.json") as f:
    data = json.load(f)

for item in tqdm(data):
    embedding = await litellm_service.embedding_llm_async(
        inputs=LiteLLMEmbeddingInput(
            text=item['question']
        )
    )
    options = [
        f"{key}. {value}"
        for key, value in item['options'].items()
    ]
    collection.add(
        ids=[str(uuid4())],
        embeddings=[embedding.embedding],
        documents=[item['question']],   # text để search
        metadatas=[{
            "options": "\n".join(options),
            "answer": item['answer'],
            "explanation": item['explanation'],
        }]
    )
    time.sleep(1)

100%|██████████| 150/150 [03:41<00:00,  1.48s/it]


In [3]:
query = "Hard Margin SVM Optimization Problem"

embeddings = await litellm_service.embedding_llm_async(
    inputs=LiteLLMEmbeddingInput(
        text=query
    )
)
results = collection.query(
    query_embeddings=[embeddings.embedding],
    n_results=3,
)

results

{'ids': [['ba055faa-e1ee-43ad-9647-836899649535',
   'd273bde6-37e8-4a11-a4ca-afcdddf3a405',
   '21a1fef9-e19b-426a-82a0-2fd80bf4ee78']],
 'embeddings': None,
 'documents': [['What is the role of the margin in SVM?',
   'What is the main purpose of a Support Vector Machine (SVM)?',
   'What is a support vector in SVM?']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'options': 'A. To separate the training and test data\nB. To measure the distance between support vectors\nC. To maximize the distance between data points and the hyperplane\nD. To reduce model complexity',
    'explanation': 'SVM aims to maximize the margin, which is the distance between the hyperplane and the nearest data points from each class, improving generalization.',
    'answer': 'To maximize the distance between data points and the hyperplane'},
   {'answer': 'To classify data points',
    'explanation': 'SVM is primarily used for classification tasks by find

In [5]:
results['documents'][0]

['What is the role of the margin in SVM?',
 'What is the main purpose of a Support Vector Machine (SVM)?',
 'What is a support vector in SVM?']

In [6]:
results['metadatas'][0]

[{'options': 'A. To separate the training and test data\nB. To measure the distance between support vectors\nC. To maximize the distance between data points and the hyperplane\nD. To reduce model complexity',
  'explanation': 'SVM aims to maximize the margin, which is the distance between the hyperplane and the nearest data points from each class, improving generalization.',
  'answer': 'To maximize the distance between data points and the hyperplane'},
 {'answer': 'To classify data points',
  'explanation': 'SVM is primarily used for classification tasks by finding the hyperplane that best separates data points into different classes.',
  'options': 'A. To reduce the number of features\nB. To classify data points\nC. To increase data size\nD. To perform clustering'},
 {'answer': 'A data point closest to the hyperplane',
  'explanation': 'Support vectors are the data points that are closest to the hyperplane and directly influence its position and orientation.',
  'options': 'A. A data

In [2]:
import pandas as pd

s1 = pd.Series([13, 31, 4])
s2 = pd.Series([4, 5, 6])

# Concatenate vertically (stack one after another)
result = pd.concat([s1, s2])
print(result)


0    13
1    31
2     4
0     4
1     5
2     6
dtype: int64
